<a href="https://colab.research.google.com/github/IlyaZutler/GBR-agents/blob/main/GBR_Agents_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import random
import numpy as np
!pip install simpy
import simpy

In [28]:
class Agent:
    def __init__(self, env, location, speed):
        self.env = env
        self.location = location
        self.base_location = location  # Место дислокации
        self.speed = speed
        self.status = 'Free'
        self.total_busy_time = 0
        self.handled_alarms = 0
        self.action = env.process(self.run())

    def run(self):
        while True:
            yield self.env.timeout(1)

    def travel_time(self, destination): # время на перемещение до
        return abs(self.location - destination) / self.speed

    def travel_to(self, destination):  # возвращает координаты Агента
        while self.location != destination:
            if self.location < destination:
                self.location = min(self.location + self.speed, destination)
            else:
                self.location = max(self.location - self.speed, destination)
            yield self.env.timeout(1)

    def handle_alarm(self, alarm):   # обработка тревоги
        self.status = 'Busy'
        # Перемещение к тревоге
        yield self.env.process(self.travel_to(alarm.location))
        # Время на тревоге
        yield self.env.timeout(alarm.complexity)
        self.total_busy_time += self.env.now - alarm.time_created
        self.handled_alarms += 1
        self.status = 'Free'  # Изменить статус на Free сразу после обработки тревоги

        # Возвращение к месту дислокации
        yield self.env.process(self.travel_to(self.base_location))

class Alarm:
    def __init__(self, env, location, complexity):
        self.env = env
        self.location = location
        self.complexity = complexity
        self.time_created = env.now

class Simulation:
    def __init__(self, num_agents, segment_length, agent_speed, alarm_rate, complexity_mean, complexity_sigma, threshold, simulation_duration):
        self.env = simpy.Environment()
        self.segment_length = segment_length
        self.agents = [Agent(self.env, location = segment_length / num_agents / 2 + i * segment_length / num_agents  , speed = agent_speed) for i in range(num_agents)]
        self.alarm_rate = alarm_rate
        self.complexity_mean = complexity_mean
        self.complexity_sigma = complexity_sigma
        self.threshold = threshold
        self.simulation_duration = simulation_duration
        self.results = {'total_alarms': 0, 'average_response_time': 0, 'alarms_above_threshold': 0, 'busy_agents': []}

    def generate_alarms(self):
        while True:
            yield self.env.timeout(random.expovariate(self.alarm_rate))
            location = random.uniform(0, self.segment_length)
            complexity = np.random.lognormal(mean=self.complexity_mean, sigma=self.complexity_sigma)
            alarm = Alarm(self.env, location, complexity)
            self.dispatch_agent(alarm)

    def dispatch_agent(self, alarm):
        free_agents = [agent for agent in self.agents if agent.status == 'Free']
        if free_agents:
            nearest_agent = min(free_agents, key=lambda agent: agent.travel_time(alarm.location))
            self.env.process(nearest_agent.handle_alarm(alarm))
        else:
            #print(f"No available agents at time {self.env.now}")
            yield self.env.timeout(0)

    def monitor(self):
        while True:
            yield self.env.timeout(1)
            busy_agents = sum(agent.status == 'Busy' for agent in self.agents)
            self.results['busy_agents'].append(busy_agents / len(self.agents))

    def report(self):
        self.results['total_alarms'] = sum(agent.handled_alarms for agent in self.agents)
        total_response_time = sum(agent.total_busy_time for agent in self.agents)
        self.results['average_response_time'] = total_response_time / self.results['total_alarms'] if self.results['total_alarms'] > 0 else 0
        self.results['alarms_above_threshold'] = sum(
            1 for agent in self.agents for _ in range(agent.handled_alarms)
            if agent.total_busy_time / agent.handled_alarms > self.threshold
        )

        print(f"Среднее время на тревоге: {np.exp(complexity_mean + 0.5 * complexity_sigma**2)}")
        print(f"Стандартное отклонение времени на тревоге: {( (np.exp(complexity_sigma**2) - 1) * np.exp(2 * complexity_mean + complexity_sigma**2))**0.5 }")

        print("Total alarms:", self.results['total_alarms'])
        print("Average response time:", self.results['average_response_time'])
        print("Alarms above threshold:", self.results['alarms_above_threshold'])

        print(f"Fraction of Delayed Alarms: {self.results['alarms_above_threshold'] / self.results['total_alarms'] if self.results['total_alarms'] else 0}")

        print("Busy agents over time:", np.mean(self.results['busy_agents']))
        for i, agent in enumerate(self.agents):
            print(f"Agent {i} handled {agent.handled_alarms} alarms, busy time: {agent.total_busy_time}")

    def run(self):
        self.env.process(self.generate_alarms())
        self.env.process(self.monitor())
        self.env.run(until=self.simulation_duration)
        self.report()

if __name__ == "__main__":
    # Настраиваемые параметры
    days = 2

    num_agents = 10
    segment_length = 80000 #80 километров
    agent_speed = 36000 / (60*60)  # 36 км/ч в секунду
    alarm_rate = 50 / (24*60*60)  # 50 тревог в день - интенсивность тревог
    complexity_mean = 0
    complexity_sigma = 1
    threshold = 15*60  # 15 минут
    simulation_duration = 24*60*60*days  # Время моделирования

    random.seed(42)
    np.random.seed(42)

    simulation = Simulation(num_agents, segment_length, agent_speed, alarm_rate, complexity_mean, complexity_sigma, threshold, simulation_duration)
    simulation.run()


Среднее время на тревоге: 1.6487212707001282
Стандартное отклонение времени на тревоге: 2.1611974158950877
Total alarms: 0
Average response time: 0
Alarms above threshold: 0
Fraction of Delayed Alarms: 0
Busy agents over time: 0.0
Agent 0 handled 0 alarms, busy time: 0
Agent 1 handled 0 alarms, busy time: 0
Agent 2 handled 0 alarms, busy time: 0
Agent 3 handled 0 alarms, busy time: 0
Agent 4 handled 0 alarms, busy time: 0
Agent 5 handled 0 alarms, busy time: 0
Agent 6 handled 0 alarms, busy time: 0
Agent 7 handled 0 alarms, busy time: 0
Agent 8 handled 0 alarms, busy time: 0
Agent 9 handled 0 alarms, busy time: 0


In [24]:
_mean = 1
_sigma = 1

np.exp(_mean + 0.5 * _sigma**2)

4.4816890703380645